# Data loading

In [3]:
IMPORT_PATH = '../Initial_Data/Unzipped'
IMPORT_ALL_PATH = IMPORT_PATH + '/*'

import pandas as pd
import numpy as np
import glob as glob

def readAllFiles():
    files = glob.glob(IMPORT_ALL_PATH)
    frames = []

    for file in files:
        df = pd.read_csv(file, index_col = 0)
        frames.append(df)

    return pd.concat(frames)

def readOneFile(url):
    return pd.read_csv(url, index_col = 0)

df = readAllFiles()

# Cleaning functions

In [17]:
import pandas as pd
import numpy as np
import glob as glob
from math import *

usaStates = [
    "AL",
    "AK",
    "AZ",
    "AR",
    "CA",
    "CO",
    "CT",
    "DE",
    "FL",
    "GA",
    "HI",
    "ID",
    "IL",
    "IN",
    "IA",
    "KS",
    "KY",
    "LA",
    "ME",
    "MD",
    "MA",
    "MI",
    "MN",
    "MS",
    "MO",
    "MT",
    "NE",
    "NV",
    "NH",
    "NJ",
    "NM",
    "NY",
    "NC",
    "ND",
    "OH",
    "OK",
    "OR",
    "PA",
    "RI",
    "SC",
    "SD",
    "TN",
    "TX",
    "UT",
    "VT",
    "VA",
    "WA",
    "WV",
    "WI",
    "WY",
]

def deleteWrongStates(df):
    print("Aantal records:", len(df))
    
    for el in df.departure_state.unique():
        if(el not in usaStates):
            df = df.drop(df[df['departure_state'] == el].index)
    print("Aantal records na verwijderen foute vertrek staat:", len(df))

    for el in df.arrival_state.unique():
        if(el not in usaStates):
            df = df.drop(df[df['arrival_state'] == el].index)
    print("Aantal records na verwijderen foute aankomst staat:", len(df))
          
    return df

def convertColumnTypes(df):
    df.departure_schedule = df.departure_schedule.astype(int)
    df.departure_delay = df.departure_delay.astype(float)
    df.arrival_schedule = df.arrival_schedule.astype(int)
    df.arrival_delay = df.arrival_delay.astype(float)
    df.arrival_actual = df.arrival_actual.astype(int)
    df.departure_actual = df.departure_actual.astype(int)
    return df

def dropMoreAdvancedDuplicates(df):
    df = df.groupby(['date', 'airline', 'airline_code', 'departure_airport', 'departure_state', 'departure_lat', 'departure_lon', 'departure_schedule', 'arrival_airport', 'arrival_state', 'arrival_lat', 'arrival_lon', 'arrival_schedule']).mean().reset_index()
    df = convertColumnTypes(df)
    if 'index' in df.columns:
        df = df.drop(['index'], axis=1) #remove old index
    return df


In [8]:
# Controleer of er records zijn die een vertrektijd hebben die vroeger is dan de aankomsttijd
# Omdat de aankomstdatum niet bijgehouden wordt, kan je niet zeker zijn of het de volgende dag is een een fout record
# Daarom controleren we ook nog of de geplande vliegtijd meer dan x aantal minuten te snel is
# We hebben geen extreme waardes gevonden en daarom geen records verwijderd
df[(df.departure_schedule > df.arrival_schedule) & ((df.arrival_delay - df.departure_delay) < -80)]

,airline,airline_code,departure_airport,departure_state,departure_lat,departure_lon,arrival_airport,arrival_state,arrival_lat,arrival_lon,departure_schedule,departure_actual,departure_delay,arrival_schedule,arrival_actual,arrival_delay
date,,,,,,,,,,,,,,,,


# Execute cleaning

In [18]:
print("Aantal records om te beginnen:", len(df))
cleaning = df

cleaning = cleaning.drop_duplicates()
print("Aantal records na verwijderen van dubbels:", len(cleaning))

cleaning = cleaning.reset_index()
print("Aantal records na nieuwe index:", len(cleaning))

cleaning = cleaning.dropna()
print("Aantal records na verwijderen lege waarden:", len(cleaning))

cleaning = deleteWrongStates(cleaning)
cleaning = convertColumnTypes(cleaning)

cleaning = dropMoreAdvancedDuplicates(cleaning)
print("Aantal records na het verwijderen van de verborgen dubbels:", len(cleaning))

cleanData = cleaning

Aantal records om te beginnen: 11401196
Aantal records na verwijderen van dubbels: 10751921
Aantal records na nieuwe index: 10751921
Aantal records na verwijderen lege waarden: 10751919
Aantal records: 10751919
Aantal records na verwijderen foute vertrek staat: 10695506
Aantal records na verwijderen foute aankomst staat: 10642322
Aantal records na het verwijderen van de verborgen dubbels: 10642032


# Export df to csv

In [19]:
EXPORT_PATH = '../Data'

def exportDfToCsvFiles(df):
    step = 1000000
    start = 0
    stop = step
    i = 0

    while start < len(df):
        if stop >= len(df):
            stop = len(df)
        fileName = EXPORT_PATH +  '/flights_2010_' + str(i) + '.csv'
        
        data = df.iloc[start:stop, 0:]
        data.to_csv( fileName, sep=',')

        start += step
        stop += step
        i += 1

exportDfToCsvFiles(cleanData)

# Functions for json

### Export everything to json

In [11]:
#creating a json file
cleanData.to_json('flights_2010_file_time_update_versie_2.json', orient='records')

### Export selected part to json

In [15]:
#selecting SouthWest Airlines
df_southWest = cleanData.loc[df['airline'] == "WN"]
#selecting Alaska Airlines
df_alaska = cleanData.loc[df['airline'] == "AS"]
#selecting Hawaiian Airlines
df_hawaiian = cleanData.loc[df['airline'] == "HA"]

In [16]:
df_southWest = df_southWest[['date', 'departure_airport','departure_state',
                          'departure_lat', 'departure_lon',
                          'arrival_airport', 'arrival_state',
                          'arrival_lat', 'arrival_lon']]
df_alaska = df_alaska[['date', 'departure_airport','departure_state',
                          'departure_lat', 'departure_lon',
                          'arrival_airport', 'arrival_state',
                          'arrival_lat', 'arrival_lon']]
df_hawaiian = df_hawaiian[['date', 'departure_airport','departure_state',
                          'departure_lat', 'departure_lon',
                          'arrival_airport', 'arrival_state',
                          'arrival_lat', 'arrival_lon']]

In [29]:
#creating a json file for SouthWest Airlines
df_southWest.to_json('SouthWest_Airlines.json', orient='records')

In [30]:
#creating a json file for Alaska Airlines
df_southWest.to_json('Alaska_Airlines.json', orient='records')

In [31]:
#creating a json file for Hawaiian Airlines
df_southWest.to_json('Hawaiian_Airlines.json', orient='records')